# 巨大なデータの取り扱い
巨大といってもここでは数千万～数億レコード（数GB～100GB未満ぐらい）のデータをPythonで処理するときについて  
色々試した結果を記す。  

In [1]:
import polars as pl
import numpy as np

# 文字列カラムの表示文字数を50文字に設定
pl.Config.set_fmt_str_lengths(50)

polars.config.Config

## ● 約5000万行 × 5列 × 1ファイル（外付けUSBに保存）の場合  
### 条件
* USBのスペック： 
    * USB3.2 Gen1  
    * 容量： 32GB
    * 他の詳細は忘れた

USBに保存してある大きめのファイルについて処理したいとき。  
データの全カラム・全レコードが解析に必要なケースは多くないと思われるので、  
LazyFrameを活用して高速化、メモリ節約を意識する。  
collect()の際はstreaming=Trueにすることで一気にメモリに読み込まずにバッチ的に読み込んで処理する。

In [2]:
! wc -l ../../../sample_data/from_HDD/00_study/sample_data/sample_big_data_1.csv

49061300 ../../../sample_data/from_HDD/big_data/sample_big_data_1.csv


In [6]:
# 読み込み条件定義
input_file = '../../../sample_data/from_HDD/00_Study/sample_data/sample_big_data_1.csv'
col_names_dtypes = {
    'datetime_col': pl.Utf8, 
    'value_1': pl.Float32, 
    'value_2': pl.Float32, 
    'value_3': pl.Float32, 
    'labels': pl.Utf8}

In [7]:
# 最初の５行だけ読み込み
(
    pl.scan_csv(input_file, has_header=False, schema=col_names_dtypes)
    .head().collect()
)

datetime_col,value_1,value_2,value_3,labels
str,f32,f32,f32,str
"""2023-11-03 13:39:36.088815""",404.69751,443.679321,87.651566,"""CCCC"""
"""2023-11-03 13:39:36.088875""",457.07785,634.807251,549.272156,"""EEEE"""
"""2023-11-03 13:39:36.088891""",620.065186,173.628784,219.745941,"""BBBB"""
"""2023-11-03 13:39:36.088906""",25.579391,143.887222,408.059845,"""BBBB"""
"""2023-11-03 13:39:36.088920""",654.784729,528.226379,1026.964355,"""BBBB"""


最初の5行を表示したりは全く問題ない。

In [7]:
%%time
# filter,with_columnsをやってみる。
(
    pl.scan_csv(input_file, has_header=False, schema=col_names_dtypes, truncate_ragged_lines=True)
    .filter(pl.col('labels') == 'EEEE')
    .with_columns(
        val_1_90tile = pl.col('value_1').quantile(0.9)
    ).collect(streaming=True)
)

CPU times: user 35.8 s, sys: 2min 41s, total: 3min 17s
Wall time: 1min 6s


datetime_col,value_1,value_2,value_3,labels,val_1_90tile
str,f32,f32,f32,str,f32
"""2023-11-03 13:39:36.088875""",457.07785,634.807251,549.272156,"""EEEE""",900.028076
"""2023-11-03 13:39:36.088961""",484.879608,265.178833,711.663208,"""EEEE""",900.028076
"""2023-11-03 13:39:36.089092""",713.928345,101.049217,836.95105,"""EEEE""",900.028076
"""2023-11-03 13:39:36.089165""",964.206848,974.932983,227.194,"""EEEE""",900.028076
"""2023-11-03 13:39:36.089313""",864.268555,748.114075,447.793671,"""EEEE""",900.028076
"""2023-11-03 13:39:36.089325""",730.194397,89.366402,932.182983,"""EEEE""",900.028076
"""2023-11-03 13:39:36.089375""",140.499466,857.064331,237.348999,"""EEEE""",900.028076
"""2023-11-03 13:39:36.089387""",719.163879,674.034851,836.932312,"""EEEE""",900.028076
"""2023-11-03 13:39:36.089461""",832.301819,503.263397,335.356476,"""EEEE""",900.028076


filterしてquantileを計算するのは1分ほどかかった。

In [10]:
%%time
# filter,with_columnsをやってみる。
(
    pl.scan_csv(input_file, has_header=False, schema=col_names_dtypes)
    .filter(pl.col('labels') == 'EEEE')
    .select(pl.col('value_1').quantile(0.9))
    .collect(streaming=True)
)

CPU times: user 21.1 s, sys: 3min 1s, total: 3min 22s
Wall time: 53.8 s


value_1
f32
900.028076


In [11]:
%%time
# streamingでfilter,with_columnsをやってみる。
(
    pl.scan_csv(input_file, has_header=False, schema=col_names_dtypes)
    .filter(pl.col('labels') == 'EEEE')
    .collect(streaming=True)
)

CPU times: user 32.8 s, sys: 2min 47s, total: 3min 20s
Wall time: 1min 6s


datetime_col,value_1,value_2,value_3,labels
str,f32,f32,f32,str
"""2023-11-03 13:39:36.088875""",457.07785,634.807251,549.272156,"""EEEE"""
"""2023-11-03 13:39:36.088961""",484.879608,265.178833,711.663208,"""EEEE"""
"""2023-11-03 13:39:36.089092""",713.928345,101.049217,836.95105,"""EEEE"""
"""2023-11-03 13:39:36.089165""",964.206848,974.932983,227.194,"""EEEE"""
"""2023-11-03 13:39:36.089313""",864.268555,748.114075,447.793671,"""EEEE"""
"""2023-11-03 13:39:36.089325""",730.194397,89.366402,932.182983,"""EEEE"""
"""2023-11-03 13:39:36.089375""",140.499466,857.064331,237.348999,"""EEEE"""
"""2023-11-03 13:39:36.089387""",719.163879,674.034851,836.932312,"""EEEE"""
"""2023-11-03 13:39:36.089461""",832.301819,503.263397,335.356476,"""EEEE"""


### ● selectしてquantileだけ求める場合

In [12]:
%%time
# filter,with_columnsをやってみる。
(
    pl.scan_csv(input_file, has_header=False, schema=col_names_dtypes)
    .select(pl.col('value_1').quantile(0.9))
    .collect(streaming=True)
)

CPU times: user 34.1 s, sys: 3min 4s, total: 3min 38s
Wall time: 51 s


value_1
f32
900.036194


In [ ]:
%%time
# filter,with_columnsをやってみる。
(
    pl.scan_csv(input_file, has_header=False, schema=col_names_dtypes)
    .select([
        pl.col('value_1').quantile(0.9),
        pl.col('value_2').quantile(0.9),
        pl.col('value_3').quantile(0.9),
    ])
    .collect(streaming=True)
)

CPU times: user 1min 19s, sys: 2min 50s, total: 4min 10s
Wall time: 1min 3s


value_1,value_2,value_3
f32,f32,f32
900.036194,900.887268,989.969238


quantileの計算を増やしても時間はそれほど変わらなかった。

In [18]:
%%time
# filter,with_columnsをやってみる。
(
    pl.scan_csv(input_file, has_header=False, schema=col_names_dtypes)
    .select([
        pl.col('value_1').quantile(i).alias(f'q_{i*100:.0f}') for i in np.linspace(0.0,1.0,10)
    ])
    .collect(streaming=True)
)

CPU times: user 3min 18s, sys: 3min 1s, total: 6min 19s
Wall time: 1min 4s


q_0,q_11,q_22,q_33,q_44,q_56,q_67,q_78,q_89,q_100
f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
0.000055,111.199875,222.284958,333.375244,444.49234,555.536194,666.622375,777.792542,888.939148,1000.0


----------------------------------------------

----------------------------------------------

----------------------------------------------

## ● 約5000万行 × 5列 × 1ファイル（内部SSDに保存）の場合  
### 条件
* 内部SSDのスペック
    * 容量：2TB
    * その他スペック忘れた

内部SSDをWSL2にマウントしてそこのパスを指定して処理する場合。

In [ ]:
! wc -l ../../../sample_data/from_HDD/00_study/sample_data/sample_big_data_1.csv

49061300 ../../../sample_data/from_HDD/big_data/sample_big_data_1.csv


In [8]:
# 読み込み条件定義
input_file = '../../../sample_data/from_HDD/00_Study/sample_data/sample_big_data_1.csv'
col_names_dtypes = {
    'datetime_col': pl.Utf8, 
    'value_1': pl.Float32, 
    'value_2': pl.Float32, 
    'value_3': pl.Float32, 
    'labels': pl.Utf8}

In [9]:
(
    pl.scan_csv(input_file, has_header=False, schema=col_names_dtypes)
    .head().collect()
)

datetime_col,value_1,value_2,value_3,labels
str,f32,f32,f32,str
"""2023-11-03 13:39:36.088815""",404.69751,443.679321,87.651566,"""CCCC"""
"""2023-11-03 13:39:36.088875""",457.07785,634.807251,549.272156,"""EEEE"""
"""2023-11-03 13:39:36.088891""",620.065186,173.628784,219.745941,"""BBBB"""
"""2023-11-03 13:39:36.088906""",25.579391,143.887222,408.059845,"""BBBB"""
"""2023-11-03 13:39:36.088920""",654.784729,528.226379,1026.964355,"""BBBB"""


In [10]:
(
    pl.scan_csv(input_file, has_header=False, schema=col_names_dtypes)
    .select(
        pl.col('value_1').mean()
    ).collect(streaming=True)
)

value_1
f32
500.024323


In [17]:
%%time
# filter,with_columnsをやってみる。
(
    pl.scan_csv(input_file, has_header=False, schema=col_names_dtypes)
    .select([
        pl.col('value_1').quantile(i).alias(f'q_{i*100:.0f}') for i in np.linspace(0.01, 1.0, 100)
    ])
    .collect(streaming=True).transpose()
)

CPU times: user 35min 53s, sys: 5min 7s, total: 41min 1s
Wall time: 4min 13s


column_0
f32
10.025609
20.031404
30.018518
40.021393
50.027016
60.030224
70.055244
80.050735
90.06752


内部SSDを指定して1%ile～100%ileの計算を実行する場合、上記の通り4分ほどかかった。  

## ● 約5000万行 × 5列 × 1ファイル（WSL2管理下に保存）の場合  
### 条件
* ー

WSL2管理下に大容量ファイルを持ってきて処理する場合。  
ディスクは食ってしまうが、これが一番早い。  
（ただし、サーバで出力されたデータの場合はこの環境まで持ってくるのにも時間がかかることは留意すること。）

In [2]:
# 読み込み条件定義
input_file = '../../../sample_data/Big_data_sample/sample_big_data_1.csv'
col_names_dtypes = {
    'datetime_col': pl.Utf8, 
    'value_1': pl.Float32, 
    'value_2': pl.Float32, 
    'value_3': pl.Float32, 
    'labels': pl.Utf8}

In [3]:
(
    pl.scan_csv(input_file, has_header=False, schema=col_names_dtypes)
    .head().collect()
)

datetime_col,value_1,value_2,value_3,labels
str,f32,f32,f32,str
"""2023-11-03 13:39:36.088815""",404.69751,443.679321,87.651566,"""CCCC"""
"""2023-11-03 13:39:36.088875""",457.07785,634.807251,549.272156,"""EEEE"""
"""2023-11-03 13:39:36.088891""",620.065186,173.628784,219.745941,"""BBBB"""
"""2023-11-03 13:39:36.088906""",25.579391,143.887222,408.059845,"""BBBB"""
"""2023-11-03 13:39:36.088920""",654.784729,528.226379,1026.964355,"""BBBB"""


In [5]:
(
    pl.scan_csv(input_file, has_header=False, schema=col_names_dtypes)
    .select(
        pl.col('value_1').mean()
    ).collect(streaming=True)
)

value_1
f32
500.024323


In [6]:
! time awk -F, '{m+=$2} END{print m/NR;}' ../../../sample_data/Big_data_sample/sample_big_data_1.csv

500.024

real	0m24.167s
user	0m22.612s
sys	0m1.551s


外付けHDDや内部SSDから読み込む場合に比べて、平均の計算にかかる時間が1/15程度になった。  
またawkで平均を求めるよりも早い。  
（上記はWSL2上のコンテナからawkを実行した場合。WSL2から直接実行するとなぜか1分以上かかった）

In [5]:
%%time
# filter,with_columnsをやってみる。
(
    pl.scan_csv(input_file, has_header=False, schema=col_names_dtypes)
    .select([
        pl.col('value_1').quantile(i).alias(f'q_{i*100:.0f}') for i in np.linspace(0.01, 1.0, 100)
    ])
    .collect(streaming=True).transpose()
)

CPU times: user 33min 11s, sys: 31.4 s, total: 33min 43s
Wall time: 2min 54s


column_0
f32
10.025609
20.031404
30.018518
40.021393
50.027016
60.030224
70.055244
80.050735
90.06752


パーセンタイルを大量に計算する場合についても40%程早くなった。